In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import networkx as nx
import utility as util

In [3]:
class Netflix_Ratings(object):
    
    def __init__(self,ratings_dir,film_title_csv):
        import pandas as pd
        import networkx as nx
        self.ratings_dir = ratings_dir
        self.film_title_csv = film_title_csv
        
        NetflixTitle = pd.read_excel(self.film_title_csv)
        NetflixTitle['Name2'] = NetflixTitle['Name2'].fillna('')
        NetflixTitle['Name3'] = NetflixTitle['Name3'].fillna('')
        NetflixTitle['Unnamed: 5'] = NetflixTitle['Unnamed: 5'].fillna('')
        NetflixTitle['Name'] = NetflixTitle.apply(self.combine, axis=1)
        self.names_df = NetflixTitle
        
        self.ID_to_name, self.name_to_ID = self.create_mappings(self.names_df)
        
    def combine(self,row):
        if row['Name2'] != '':
            row['Name'] = str(row['Name']) + ', ' + str(row['Name2'])
        if row['Name3'] != '':
            row['Name'] = row['Name'] + ', ' + str(row['Name3'])
        if row['Unnamed: 5'] != '':
            row['Unnamed: 5'] = row['Name'] + ', ' + str(row['Unnamed: 5'])
        return row['Name']
    
    def create_mappings(self,names_df):
        IDs = list(names_df['ID'])
        names = list(names_df['Name'].str.lower())
        ID_to_name = dict(zip(IDs,names))
        name_to_ID = dict(zip(names,IDs))
        return ID_to_name, name_to_ID
    
    def create_utility_matrix(self,G):
        '''
        Given a network G, this method will construct the utility matrix for the movies present in 
        the nodeset of G that are also within the ratings listed here.
        '''
        import csv
        import numpy as np
        import scipy.sparse as ss
        
        titles_in_matrix = [i for i in G.nodes()]
        ids_in_matrix = [self.name_to_ID[x] for x in titles_in_matrix]
        
        # Loop through the files for each movie, compile the ratings for each movie, and 
        # get all of the users who rated each movie. 
        ratings_dict = {}
        users = []
        for title in titles_in_matrix[:]: #REMOVE THROTTLE
            # Get the Netflix id of this movie, and the title of the review file.
            filename = f"{self.ratings_dir}mv_{self.name_to_ID[title]:07}.txt"
            # Build a nested dictionary, where the outer key is the title of the movie,
            # the inner key is the numeric identifier of the user, and the value is the
            # rating.
            ratings = {}
            with open(filename,'r') as f:
                reader = csv.reader(f)
                for i,row in enumerate(reader):
                    if i == 0:
                        continue
                    else:
                        users.append(row[0])
                        ratings[row[0]] = row[1]
            ratings_dict[title] = ratings
        users = list(set(users))
        
        # Make mappings for the movie title and user to index
        title_to_index = dict(zip(titles_in_matrix,range(len(titles_in_matrix))))
        index_to_title = dict(zip(range(len(titles_in_matrix)),titles_in_matrix))
        user_to_index = dict(zip(users,range(len(users))))
        index_to_user = dict(zip(range(len(users)),users))

        # Build the utility matrix. [j,i] where j is user and i is movie.
        um = np.full((len(users),len(titles_in_matrix)),0)
        um = ss.lil_matrix(um)
        for title in ratings_dict:
            i = title_to_index[title]
            for user in ratings_dict[title]:
                j = user_to_index[user]
                um[j,i] = ratings_dict[title][user]
        um = ss.csr_matrix(um)
        
        self.um = um
        self.index_to_title = index_to_title
        self.index_to_user = index_to_user
        self.title_to_index = title_to_index
        self.user_to_index = user_to_index

G = util.parse_nodes_edge_file('AllActorG.net')
ratings = Netflix_Ratings('training_set/','Netflix-Dataset/movie_titles_test.xls')
ratings.create_utility_matrix(G)

In [4]:
import pickle

In [5]:
import pickle
pickle.dump( ratings, open( "saveratings.p", "wb" ) )

In [11]:
ratingsTest = pickle.load( open( "saveratings.p", "rb" ) )
ratings = ratingsTest

In [6]:
ratingsTest.um.shape

NameError: name 'ratingsTest' is not defined

In [7]:
# create_utility_matrix(G)

In [490]:
class Network_Recommender(object):
    
    def __init__(self,G,ratings,test_size=0.2):
        self.G = G # graph
        self.ratings = ratings # ratings object
        self.U = self.ratings.um
        self.test_size = test_size
        
    def train_test_split(self,user_i,):
        """
        Given a user-vector at index user_i, will return with test_size percent as 0s.
        Will also return the indices of the test.
        
        Returns:
            - j,i,v for the sparse vector being tested, after replaced with testing points of 0 rating
            - list of tuples that contain the indicies (j,i) of the test elements from self.U
        """
        import scipy.sparse as ss
        import numpy as np
        
        test_size=self.test_size
                
        j,i,v = ss.find(self.U[user_i,:])
        np.random.seed(42)
        rand = np.random.uniform(size=j.size)
        test_i = []
        
        x = 0
        for jj,ii,r in zip(j,i,rand):
            if r < test_size:
                v[x] = 0
                test_i.append((jj,ii))
            x+=1
                
        #print(Uc)
        #print(test_i)
        
        return j,i,v,test_i
        

        
    def match_nodes(self,x,y):
        if x['rating'] == y['rating']:
            return True
        else:
            return False
        
    def ICA(self,user_i,verbose=True):
        """
        Parameters:
            - user_i: index of the user to perform ICA with
        Returns:
            - vector of predictions for user_i
            - mae score of this vector
        """
        import networkx as nx
        import numpy as np
        
        j,i,v,test_i = self.train_test_split(user_i)
        mean_rating = np.average(v)
        G_ = self.G.copy()
        
        # Set initial ratings from the user. These include any testing points.
        for x,ii in enumerate(i):
            if v[x] != 0:
                G_.nodes[self.ratings.index_to_title[ii]]['rating'] = v[x]
                
        # Initialize the rest of the nodes:
        for n in G_.nodes():
            if 'rating' not in G_.nodes[n]:
                G_.nodes[n]['rating'] = 0

                
        # Start algorithm by initializing the nest time step and a time tracker.
        G_plus = G_.copy()
        num_zeros_ = -999
        t = 0
        still_changing = True
        
        
        while still_changing:
            if verbose:
                print(f"timestep: {t}")
            
            # Recall that 'n' in this case is actually the title of the movie
            for n in G_.nodes():
                if G_.nodes[n]['rating'] == 0:
                # To get the edges of each node, use G_[], to get the node props, use G_.nodes[]
                    nns = G_[n]
                    # Check each neighbor for ratings, and store them.
                    nn_ratings = []
                    nn_weights = []
                    for nn in nns:
                        if G_.nodes[nn]['rating'] != 0:
                            nn_ratings.append(G_.nodes[nn]['rating'])
                            nn_weights.append(G_[n][nn]['weight'])
                    # If any ratings existed in the neighbors, update this node with the avg rating between neighbors.
                    if nn_ratings:
                        G_plus.nodes[n]['rating'] = np.average(nn_ratings,weights=nn_weights)

            # Check if the graph is still changing:
            ratings_list = list(nx.get_node_attributes(G_plus,'rating').values())
            num_zeros_plus = ratings_list.count(0.0)
            if verbose:
                print(f"number of zeroes remaining: {num_zeros_plus}")
            if num_zeros_plus == num_zeros_:
                still_changing = False
            
            # Update pparameters for next iteration.
            t+=1
            num_zeros_ = num_zeros_plus
            G_ = G_plus.copy()
            
        # END ITERATION
        
        # Fill in all remaining zeros with the average rating from this user
        # and convert them back into a vector.
        outvec = np.zeros(self.U.shape[1])
        for n in G_.nodes():
            if G_.nodes[n]['rating'] == 0:
                G_.nodes[n]['rating'] = mean_rating
            outvec[self.ratings.title_to_index[n]] = G_.nodes[n]['rating']
            
        # Finally, compute MAE if there are any test_i
        if test_i:
            err = []
            for indices in test_i:
                #print(indices)
                #print(self.U[user_i,indices[1]], outvec[indices[1]])
                err.append(self.U[user_i,indices[1]] - outvec[indices[1]])
                #print(self.U[indices[0],indices[1]],outvec[indices[1]],err)
            mae = np.sum(np.abs(np.array(err)))/len(test_i)
            
        else:
            mae = np.nan
                    
            
        return outvec, mae
    
    
    def Absorb(self,user_i,verbose=True):
        """
        Parameters:
            - user_i: index of the user to perform ICA with
        Returns:
            - vector of predictions for user_i
            - mae score of this vector
        """
        import networkx as nx
        import numpy as np
        import random
        
        
        j,i,v,test_i = self.train_test_split(user_i)
        mean_rating = np.average(v)
        G_ = self.G.copy()
        
        for x,ii in enumerate(i):
            if v[x] != 0:
                G_.nodes[self.ratings.index_to_title[ii]]['rating'] = v[x]
        
        for n in G_.nodes():
            if 'rating' not in G_.nodes[n]:
                G_.nodes[n]['rating'] = 0
        
        G_plus = G_.copy()
        num_zeros_ = -999
        t = 0
        still_changing = True
        
        while still_changing:
            if verbose:
                print(f"timestep: {t}")
            
            # Recall that 'n' in this case is actually the title of the movie
            for n in G_.nodes():
                #print(n)
                #if the node is rated at 0 we want to perform the absorbing method
                if G_.nodes[n]['rating'] == 0:
                    
                    absNodeRatings = []
                    #we want to try this method 'ki' times for each node
                    ki = 50
                    for kii in range(ki):
                        newN = n
                        # To get the edges of each node, use G_[], to get the node props, use G_.nodes[]
                        absorbedRatings = []
                        still_looking = 0
                        while True:
                            nns = G_[newN]
                            # Check each neighbor for ratings, and store them.
                            neighbors = []
                            #nns is the nodes that are connected to our current node
                            for nn in nns:
                                #what im doing here is getting the weight of the edge
                                #then adding the node to a list 'weight' times
                                #Doing this for each edge will make a list with nodes to travel to
                                #with the proportion of the weight
                                weight= G_[newN][nn]['weight']
                                for i in range(weight):
                                    neighbors.append(nn)
                                    
                            #was getting error if neighbors was empty
                            if neighbors:
                                randNode = random.choice(neighbors)
                            else:
                                absorbedRatings.append(mean_rating)
                            if G_.nodes[randNode]['rating'] != 0:
                                absorbedRatings.append(G_.nodes[nn]['rating'])
                                break
                            else:
                                newN = randNode
                                still_looking += 1

                            if still_looking >= 10:
                                absorbedRatings.append(mean_rating)
                                break
                    G_plus.nodes[n]['rating'] = np.mean(absorbedRatings)
        
            ratings_list = list(nx.get_node_attributes(G_plus,'rating').values())
            num_zeros_plus = ratings_list.count(0.0)
            if num_zeros_plus == num_zeros_:
                still_changing = False
                
            t+=1
            num_zeros_ = num_zeros_plus
            G_ = G_plus.copy()
            
            
        # END ITERATION
        
        # Fill in all remaining zeros with the average rating from this user
        # and convert them back into a vector.
        outvec = np.zeros(self.U.shape[1])
        for n in G_.nodes():
            if G_.nodes[n]['rating'] == 0:
                G_.nodes[n]['rating'] = mean_rating
            outvec[self.ratings.title_to_index[n]] = G_.nodes[n]['rating']
            
        if test_i:
            err = []
            for indices in test_i:
                #print(indices)
                #print(self.U[user_i,indices[1]], outvec[indices[1]])
                err.append(self.U[user_i,indices[1]] - outvec[indices[1]])
                #print(self.U[indices[0],indices[1]],outvec[indices[1]],err)
            mae = np.sum(np.abs(np.array(err)))/len(test_i)
            
        else:
            mae = np.nan
        
        return outvec, mae
    
    
    def Hitting_time(self,user_i,verbose=True,max_steps=50.0,num_walkers=100,lowest_k=3):
        """
        An implementation of hitting time, as calculated from 
        random walk. The random walk is limited to max_steps, and any
        nodes that are not reached at least once in max_steps are assigned 
        a hitting time of max_steps. The average hitting time for each node is
        then calculated based off of the lowest_k lowest hitting time already-
        rated movies.
        
        This implementation starts from the test nodes. Random walkers then traverse
        the graph, and update the hitting time of each labeled (i.e. rated) node as they 
        reach it. If the movie being tested is isolated and has no neighbors, then the
        average user's rating is used instead.
        
        Users that have less than lowest_k test movies are ignored.
        
        Returns a tuple of test indices (from self.U), predictions, and residuals; and 
        then the MAE.
        """
        import networkx as nx
        import numpy as np
        import random
        
        # First, get all of the defined and undefined nodes, and make ratings
        # for each (if they are already defined, use those. Else, use 0 and calculate)
        # If the rating is already defined, save it's node to be iterated through for 
        # the hitting time algo.
        j,i,v,test_i = self.train_test_split(user_i)
        mean_rating = np.average(v)
        G_ = self.G.copy()
        
        if len(test_i) >= lowest_k:
        
            labeled_nodes = []

            # Initialize the network ratings:

            for x,ii in enumerate(i):
                if v[x] != 0:
                    G_.nodes[self.ratings.index_to_title[ii]]['rating'] = v[x]
                    labeled_nodes.append(self.ratings.index_to_title[ii])

            for n in G_.nodes():
                if 'rating' not in G_.nodes[n]:
                    G_.nodes[n]['rating'] = 0

            # Initialize the data objects to capture the hitting time stats.

            # First, make an array that will store all of the hitting times from each
            # labeled node for each unlabeled node. This matrix will therefore be n_test nodes
            # x n_labeled nodes.

            test_nodes = [i[1] for i in test_i]
            test_node_names = [self.ratings.index_to_title[i] for i in test_nodes]

            # Make a mapping for the random walk arrays.
            tnn_to_index = dict(zip(test_node_names,range(len(test_node_names))))
            index_to_tnn = dict(zip(range(len(test_node_names)),test_node_names))
            ln_to_index = dict(zip(labeled_nodes,range(len(labeled_nodes))))
            index_to_ln = dict(zip(range(len(labeled_nodes)),labeled_nodes))

            hitting_times_matrix = np.full((len(test_node_names),len(labeled_nodes)),max_steps)

            # Now, make a dictionary that will have an entry for each of the unlabeled nodes.
            # Each of these entries will have a list of hitting times from a single test node.
            for k,tnn in enumerate(test_node_names[:]):
                
                # Keep track of just the walks spawning from this labeled node (ln)
                hitting_times_from_tnn = np.full((len(labeled_nodes),num_walkers),max_steps)
                
                # Check to make sure that there exists a path out of this movie. If not,
                # just return the inner matrix of all 0s.
                if len(list(G.neighbors(tnn))) > 0:

                    # Start looping over walkers, taking up to max_steps - worth of steps.
                    # THIS WOULD BE IDEAL PLACE FOR PARALLELIZATION.
                    for x in range(num_walkers)[:]:

                        # DEBUG:
                        path = []

                        step_i = 1
                        current_node = tnn

                        while step_i < max_steps:
                            #DEBUG:
                            path.append(current_node)

                            # Get the neighbors to go to and make a probability list to pull from
                            # based off of their weight. Choose the next node.

                            nodes = [n for n in G_[current_node]]
                            weights = [G_[current_node][n]['weight'] for n in G_[current_node]]

                            # Check to make sure that there exists a path out of this movie. If not,
                            # just return the inner matrix of all 0s.
                            try:
                                next_node = random.choices(nodes,weights,k=1)[0]
                            except Exception as e:
                                print(current_node)
                                print(e)
                                print(nodes)
                                print(weights)
                                print(path)
                                raise SystemExit()

                            # Now, update the first hitting time if haven't already
                            if current_node in labeled_nodes:
                                if hitting_times_from_tnn[ln_to_index[current_node],x] == max_steps:
                                    hitting_times_from_tnn[ln_to_index[current_node],x] = step_i

                            # Update for next random step
                            step_i += 1
                            current_node = next_node
                            
                else: 
                    
                    print(f"{tnn} has no neighbors...skipping")
                    hitting_times_from_tnn.fill(0.0)

                # Take the average of hitting time for each unlabeled node. Then, add this column 
                # vector to the hitting time matrix, which keeps track of the average hitting time
                # for each unlabeled node from each labeled node.

                #print(np.nonzero(hitting_times_from_ln==9)) # <- find indicies of nonzero
                mean_hitting_times_from_tnn = np.mean(hitting_times_from_tnn,axis=1)
                #print(mean_hitting_times_from_tnn)
                hitting_times_matrix[tnn_to_index[tnn],:] = mean_hitting_times_from_tnn
                #print(np.nonzero(hitting_times_matrix!=max_steps))
                if verbose:
                    if k % 20 == 0:
                        print(f"{k} / {len(test_node_names)}")

            # Now, make predictions on all of the unlabeled movies (we will only use the ones
            # that we need for the mae testing here. We would need a more powerful machine and 
            # implemented parallelization to handle much more than that)
            # print(hitting_times_matrix)

            # get the movies from test_i
            # test_movies = [ratings.index_to_title[m[1]] for m in test_i]
            # now get the indices in the hitting time matrix for those movies
            # test_ht_indices = [un_to_index[title] for title in test_movies]

            # Get the predicted score as an average of the lowest_k smallest
            # hitting times. Then, compare to the utility matrix truth.
            predictions = []
            residuals = []
            for test_j in range(hitting_times_matrix.shape[0])[:]:
                #movie = index_to_un[un_j]
                #utility_i = ratings.title_to_index[movie]
                #print(test_i[x][1]) <--this gets the index for the utility matrix
                test_vector = hitting_times_matrix[test_j,:].squeeze()
                
                # Catch the movies with no neighbors, and simply predicted the average:
                if np.sum(test_vector) < 0.0001:
                    predicted_score = mean_rating
                else:
                    closest_hits = self.get_smallest_indices(test_vector,lowest_k)
                    closest_hits_movies = [index_to_ln[i] for i in closest_hits]
                    closest_hits_scores = np.array([G_.nodes[i]['rating'] for i in closest_hits_movies])
                    predicted_score = np.mean(closest_hits_scores)
                    
                predictions.append(predicted_score)
                residuals.append(self.U[user_i,test_i[test_j][1]]-predicted_score)
            mae = np.mean(np.abs(residuals))

            # Gather some other information
            other = (test_i,predictions,residuals)
        
        else:
            mae = np.nan
            other = None

        # return
        return other,mae
            
                
        
    def get_smallest_indices(self,v,k):
        """
        Returns the indices of the smallest k elements of 1d array v.
        They are unsorted.
        """
        return np.argpartition(v,k)[:k]
            
    
    
    def testing_algo(self,algorithm,number_of_users=100):
        import random
        import numpy as np
        
        users = list(range(self.U.shape[0]))
        sample = random.sample(users,k=number_of_users)
        
        ms = []
        if algorithm == 'ICA':
            for x,j in enumerate(sample):
                #print(x,j)
                print(f"Sample: {x}")
                o,m = self.ICA(j,verbose=False)
                ms.append(m)

            return np.nanmean(ms)
        
        if algorithm == 'Absorb':
            for x,j in enumerate(sample):
                #print(x,j)
                print(f"Sample: {x}")
                o,m = self.Absorb(j,verbose=False)
                ms.append(m)

            return np.nanmean(ms)
        
        if algorithm == 'HT':
            for x,j in enumerate(sample):
                #print(x,j)
                print(f"Sample: {x}")
                o,m = self.Hitting_time(j,verbose=True)
                ms.append(m)

            return np.nanmean(ms)
        
        
            
        
        
    


In [459]:
#G = util.parse_nodes_edge_file('AllActorG.net')

In [453]:
network = Network_Recommender(G,ratings)
network.Hitting_time(4,num_walkers=100)

0 / 45
1 / 45
2 / 45
3 / 45
4 / 45
5 / 45
6 / 45
7 / 45
8 / 45
9 / 45
10 / 45
11 / 45
12 / 45
13 / 45
14 / 45
15 / 45
16 / 45
17 / 45
18 / 45
19 / 45
20 / 45
21 / 45
22 / 45
23 / 45
24 / 45
25 / 45
26 / 45
27 / 45
28 / 45
29 / 45
30 / 45
31 / 45
32 / 45
33 / 45
34 / 45
35 / 45
36 / 45
37 / 45
38 / 45
39 / 45
40 / 45
41 / 45
42 / 45
43 / 45
44 / 45
0.6740740740740742


In [491]:
G = util.parse_nodes_edge_file('AllActorG.net')
network = Network_Recommender(G,ratings)
#print(network.testing_algo(algorithm='ICA', number_of_users=100))
#print(network.testing_algo(algorithm='Absorb', number_of_users=100))
print(network.testing_algo(algorithm='HT', number_of_users=10))

Sample: 0
0 / 11
Sample: 1
0 / 37
20 / 37
Sample: 2
0 / 18
Sample: 3
0 / 60
20 / 60
40 / 60
Sample: 4
Sample: 5
0 / 6
Sample: 6
0 / 95
20 / 95
40 / 95
60 / 95
80 / 95
Sample: 7
0 / 13
Sample: 8
0 / 6
Sample: 9
0 / 7
0.7055252499696942
